In [1]:
# import dependencies
import pandas as pd
import matplotlib as plt
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [2]:
# setup SQL database connection
Base = automap_base()
engine = create_engine('sqlite:///../data/fbdata.sqlite')
Base.metadata.create_all(engine)
Base.prepare(engine, reflect=True)
session = Session(engine)

In [3]:
# get draft data
draftDf = pd.read_sql_query("SELECT * FROM draftHistory WHERE Pos = 'QB'", con=engine)
print(draftDf.columns)
del draftDf['CollegeStats']
draftDf

Index(['DraftYear', 'Rnd', 'Pick', 'Tm', 'Player', 'Pos', 'Age', 'To', 'AP1',
       'PB', 'St', 'College/Univ', 'CollegeStats'],
      dtype='object')


,DraftYear,Rnd,Pick,Tm,Player,Pos,Age,To,AP1,PB,St,College/Univ
0,1987,1,1,TAM,Vinny Testaverde,QB,23,2007,0,2,15,Miami (FL)
1,1987,1,6,STL,Kelly Stouffer,QB,23,1992,0,0,0,Colorado St.
2,1987,1,13,ATL,Chris Miller,QB,22,1999,0,1,7,Oregon
3,1987,1,26,CHI,Jim Harbaugh,QB,23,2000,0,1,10,Michigan
4,1987,3,64,HOU,Cody Carlson,QB,23,1994,0,0,0,Baylor
...,...,...,...,...,...,...,...,...,...,...,...,...
430,2020,6,189,JAX,Jake Luton,QB,None,None,0,0,0,Oregon St.
431,2020,7,224,TEN,Cole McDonald,QB,None,None,0,0,0,Hawaii
432,2020,7,231,DAL,Ben DiNucci,QB,None,None,0,0,0,James Madison
433,2020,7,240,NOR,Tommy Stevens,QB,23,None,0,0,0,Mississippi St.


In [4]:
# get combine stats
combineStats = pd.read_sql_query("SELECT * FROM combineStats", con=engine)
print(combineStats.columns)
combineStats['DraftYear'] = combineStats['year'].astype('int')
combineStats['Player'] = combineStats['name']
del combineStats['year']
del combineStats['name']
del combineStats['college']
combineStats

Index(['year', 'name', 'college', 'position', 'height', 'weight', '40yard',
       'vertleap', 'broadjump', 'shuttle', '3cone'],
      dtype='object')


,position,height,weight,40yard,vertleap,broadjump,shuttle,3cone,DraftYear,Player
0,QB,75.0,225.0,4.6900,35.000000,125.000000,4.510000,7.33000,2020,Kelly Bryant
1,QB,76.0,216.0,4.8776,30.460803,108.912046,4.354681,7.20592,2020,Joe Burrow
2,QB,76.0,225.0,5.0000,28.000000,109.000000,4.370000,7.13000,2020,Kevin Davidson
3,QB,78.0,227.0,4.8900,27.500000,110.000000,4.750000,7.50000,2020,Jacob Eason
4,QB,74.0,220.0,5.0100,30.000000,111.000000,4.510000,7.27000,2020,Jake Fromm
...,...,...,...,...,...,...,...,...,...,...
708,QB,75.6,207.0,5.1500,26.500000,104.000000,4.630000,7.20592,1987,Loren Snyder
709,QB,75.0,212.0,4.9500,28.000000,108.000000,4.510000,7.20592,1987,Kelly Stouffer
710,QB,71.3,187.0,4.8776,30.460803,108.912046,4.354681,7.20592,1987,Kevin Sweeney
711,QB,76.3,213.0,4.7500,30.460803,108.912046,4.354681,7.20592,1987,Vinny Testaverde


In [5]:
# start making model dataframe
modelDF = pd.merge(draftDf,combineStats,on=['DraftYear','Player'])
modelDF = modelDF.rename(columns={'College/Univ':'school'})
modelDF

,DraftYear,Rnd,Pick,Tm,Player,Pos,Age,To,AP1,PB,St,school,position,height,weight,40yard,vertleap,broadjump,shuttle,3cone
0,1987,1,1,TAM,Vinny Testaverde,QB,23,2007,0,2,15,Miami (FL),QB,76.3,213.0,4.7500,30.460803,108.912046,4.354681,7.20592
1,1987,1,6,STL,Kelly Stouffer,QB,23,1992,0,0,0,Colorado St.,QB,75.0,212.0,4.9500,28.000000,108.000000,4.510000,7.20592
2,1987,1,13,ATL,Chris Miller,QB,22,1999,0,1,7,Oregon,QB,73.4,190.0,4.7400,30.500000,117.000000,4.350000,7.20592
3,1987,1,26,CHI,Jim Harbaugh,QB,23,2000,0,1,10,Michigan,QB,74.3,203.0,4.8776,30.460803,108.912046,4.354681,7.20592
4,1987,3,64,HOU,Cody Carlson,QB,23,1994,0,0,0,Baylor,QB,75.0,191.0,4.8200,29.500000,111.000000,4.450000,7.20592
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350,2020,4,125,NYJ,James Morgan,QB,23,None,0,0,0,Florida International,QB,76.0,213.0,4.8900,29.000000,112.000000,4.640000,7.51000
351,2020,5,167,BUF,Jake Fromm,QB,None,None,0,0,0,Georgia,QB,74.0,220.0,5.0100,30.000000,111.000000,4.510000,7.27000
352,2020,6,189,JAX,Jake Luton,QB,None,None,0,0,0,Oregon St.,QB,79.0,229.0,4.8776,30.460803,108.912046,4.354681,7.20592
353,2020,7,224,TEN,Cole McDonald,QB,None,None,0,0,0,Hawaii,QB,76.0,220.0,4.5800,36.000000,121.000000,4.520000,7.13000


In [6]:
# CFB Stats
CFBStats = pd.read_sql_query("SELECT * FROM CFBStats", con=engine)
CFBStats = CFBStats.rename(columns={'name':'Player'})
CFBStats['DraftYear'] = CFBStats['DraftYear'].astype('int')
CFBStats = CFBStats[['games','rushAvg','plays','ydsFromScrimAvg','compPct','ydsPerAtt','adjYdsPerAtt','effRate','Player','DraftYear']]
CFBStats


,games,rushAvg,plays,ydsFromScrimAvg,compPct,ydsPerAtt,adjYdsPerAtt,effRate,Player,DraftYear
0,45.0,-2.6,126,-2.4,61.3,9.0,8.7,152.9,Vinny Testaverde,1987
1,32.0,-3.4,141,-3.4,56.8,7.0,6.1,120.2,Kelly Stouffer,1987
2,36.0,0.9,183,1.0,55.2,6.6,5.7,116.4,Chris Miller,1987
3,49.0,1.4,218,1.4,62.4,8.8,8.2,145.6,Jim Harbaugh,1987
4,33.0,2.2,195,2.2,51.9,7.7,6.4,122.2,Cody Carlson,1987
...,...,...,...,...,...,...,...,...,...,...
365,29.0,-1.1,116,-1.1,62.2,7.1,7.3,135.6,Jake Luton,2020
366,33.0,3.5,251,3.5,61.4,8.0,8.3,146.8,Cole McDonald,2020
367,11.0,2.5,57,2.5,54.5,6.6,5.5,113.6,Ben DiNucci,2020
368,29.0,5.6,173,5.5,59.9,7.2,7.4,139.1,Tommy Stevens,2020


In [7]:
# merge on CFB Stats
modelDF = pd.merge(modelDF, CFBStats,on=['Player','DraftYear'])
modelDF.columns

Index(['DraftYear', 'Rnd', 'Pick', 'Tm', 'Player', 'Pos', 'Age', 'To', 'AP1',
       'PB', 'St', 'school', 'position', 'height', 'weight', '40yard',
       'vertleap', 'broadjump', 'shuttle', '3cone', 'games', 'rushAvg',
       'plays', 'ydsFromScrimAvg', 'compPct', 'ydsPerAtt', 'adjYdsPerAtt',
       'effRate'],
      dtype='object')

In [8]:
# define pro success
modelDF['PB'] = modelDF['PB'].astype(int)
modelDF['St'] = modelDF['St'].astype(int)
for index, row in modelDF.iterrows():
    modelDF.at[index, 'playsPerGame'] = row.plays / row.games
    modelDF.at[index, 'lbsPerIn'] = row.weight / row.height
    if row.PB > 0 or row.St > 5:
        modelDF.at[index, 'success'] = 1
        
    else:
        modelDF.at[index, 'success'] = 0        
        
print(modelDF[modelDF['success'] == 1].count()[0], modelDF.count()[0])

61 315


In [9]:
# check output dataframe
modelDF = modelDF[['Player', 'DraftYear', 'Rnd', 'Pick', 'Tm', 'Pos', 'Age', 'To', 'AP1',\
       'PB', 'St', 'school', 'position', 'height', 'weight', 'lbsPerIn', '40yard',\
       'vertleap', 'broadjump', 'shuttle', '3cone', 'games', 'playsPerGame','plays',  \
       'rushAvg','ydsFromScrimAvg', 'compPct', 'ydsPerAtt', 'adjYdsPerAtt',\
       'effRate','success']]
modelDF

,Player,DraftYear,Rnd,Pick,Tm,Pos,Age,To,AP1,PB,...,games,playsPerGame,plays,rushAvg,ydsFromScrimAvg,compPct,ydsPerAtt,adjYdsPerAtt,effRate,success
0,Vinny Testaverde,1987,1,1,TAM,QB,23,2007,0,2,...,45.0,2.800000,126,-2.6,-2.4,61.3,9.0,8.7,152.9,1.0
1,Kelly Stouffer,1987,1,6,STL,QB,23,1992,0,0,...,32.0,4.406250,141,-3.4,-3.4,56.8,7.0,6.1,120.2,0.0
2,Chris Miller,1987,1,13,ATL,QB,22,1999,0,1,...,36.0,5.083333,183,0.9,1.0,55.2,6.6,5.7,116.4,1.0
3,Jim Harbaugh,1987,1,26,CHI,QB,23,2000,0,1,...,49.0,4.448980,218,1.4,1.4,62.4,8.8,8.2,145.6,1.0
4,Cody Carlson,1987,3,64,HOU,QB,23,1994,0,0,...,33.0,5.909091,195,2.2,2.2,51.9,7.7,6.4,122.2,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310,James Morgan,2020,4,125,NYJ,QB,23,None,0,0,...,42.0,3.095238,130,-1.3,-1.3,57.2,7.1,6.9,128.8,0.0
311,Jake Fromm,2020,5,167,BUF,QB,None,None,0,0,...,43.0,3.116279,134,0.3,0.3,63.3,8.4,9.1,156.2,0.0
312,Jake Luton,2020,6,189,JAX,QB,None,None,0,0,...,29.0,4.000000,116,-1.1,-1.1,62.2,7.1,7.3,135.6,0.0
313,Cole McDonald,2020,7,224,TEN,QB,None,None,0,0,...,33.0,7.606061,251,3.5,3.5,61.4,8.0,8.3,146.8,0.0


In [13]:
# split dataframe into modeltraintest and modelrecent
modelTrainValidate = modelDF[modelDF['DraftYear'] <= 2014]
modelRecent = modelDF[modelDF['DraftYear'] > 2014]

In [14]:
# check some numbers 
print(modelTrainValidate[modelTrainValidate['success'] == 1].count()[0], modelTrainValidate.count()[0])
print(modelRecent[modelRecent['success'] == 1].count()[0], modelRecent.count()[0])

53 250
8 65


In [15]:
# commit to sqlite database
modelTrainTest.to_sql('modelTrainValidate', con=engine, if_exists='replace', index=False)
modelRecent.to_sql('modelRecent', con=engine, if_exists='replace', index=False)